In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tifffile as tf
from scipy.ndimage.filters import gaussian_filter1d as gaussian
from glob import glob
from scipy.signal import savgol_filter, medfilt
import matplotlib.ticker as ticker
from scipy.optimize import curve_fit

In [2]:
# 1. load data into pandas data frame

# 2. convert data from I(t) to c_i(t) via equation for G quadruplex binding
    # - find t0
    # - 
# 3. calculate dc_i/dt for each GUV

# 4. calculate background intensity and convert this to c_0(t) via g quadruplex binding model

# 5. plot dc_/dt/A_GUV vs c_0-c_i (expect to see a positive linear relationship)


# as data includes actual G4 background for GUV permeability and later Gramicidin A experiments, start with these experiments


In [3]:
App = 3.16 #um^2

# G quadruplex model binding


def exp_I(c, k = 8.8,A = 0.39):
    return  A+ (1-A)/(k_1*c**2+1)

def inv(I, k = 8.8,A = 0.39):
    
    if I > 1:
        return 0
    
    elif I < A:
        return 1
        
    else:
        return np.sqrt((1/k)*((1-A)/(I-A)-1))


#smoothing configs

sigma = 30




# Calibration model fit


## **If you have already fitted a calibration function and saved the parameters skip this section**


In [4]:
# finding a suitable calibration

calib_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/RelIntensG41Concentration.csv')

In [5]:
raw_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/1uM/Intensities0-2mM.csv')
n = 512**2

print(raw_data['StdDev']/np.sqrt(n))

dI = raw_data['StdDev']/np.sqrt(n)
drelI = dI/raw_data['Mean']

drelI = drelI*10

rel_errorK = 0.02


0    2.244234
1    2.140793
2    1.921438
3    1.709535
4    1.638162
5    1.617246
Name: StdDev, dtype: float64


In [6]:
%matplotlib qt
I= np.linspace(1,0.2,100)

fig, ax = plt.subplots(1,1)

ax.scatter(calib_data['Rel. Intes'],calib_data['K conc [mM]'],marker = '+',s = 5)

legend = []

# fit a polynomial


fit = np.polyfit(-np.log(calib_data['Rel. Intes'].iloc[:-1]),calib_data['K conc [mM]'].iloc[:-1],3)

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
        print(len(fit)-i)
    return c



ax.plot(I,fit_func(I,fit))

legend.append('Calibration')

def model_fit(cK,kb,c):
    return c + (1-c)/(kb*cK**2+1)

params = curve_fit(model_fit,calib_data['K conc [mM]'],calib_data['Rel. Intes'], p0 = [8.8,0.45])

cK_bar = np.linspace(0,2)
I_bar = model_fit(cK_bar,params[0][0],params[0][1])

ax.set_xlim([0.35,1.05])
ax.plot(I_bar,cK_bar,label = 'Model')
legend.append('Model')

l  = plt.errorbar(calib_data['Rel. Intes'],calib_data['K conc [mM]'],rel_errorK*calib_data['K conc [mM]'],drelI,c = 'k')

ax.set_xticks([0.4,0.6,0.8,1])
ax.set_xticklabels([0.4,0.6,0.8,1],fontsize  = 15)

ax.set_yticks([0,1,2])
ax.set_yticklabels([0,1,2],fontsize = 15)
ax.set_ylim([-0.1,2.1])
plt.tick_params(direction = 'in',top = True,bottom = True,right = True, left = True,length = 5)
ax.set_xlabel('Rel Intensity [a.u]',fontsize = 18)
ax.set_ylabel('[K]/[mM]',fontsize = 18)
l[0].remove()
plt.legend(legend)



3
2
1
0


In [4]:
fit_dict = {'Fitting Params':fit}
df_fit = pd.DataFrame(fit_dict)

df_fit.to_csv('../Calibrations/Calibration_approx_fit_params.csv')

NameError: name 'fit' is not defined

# Load calibration function parameters and define function


In [5]:
fitsdf = pd.read_csv('../Calibrations/Calibration_approx_fit_params.csv')

fit = fitsdf['Fitting Params']

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
        print(len(fit)-i)
    return c

In [39]:

t0 = 4
t1 = 80
dt = 9/60

In [7]:
# load 260pM Gramicidin A experiment

dfI = pd.read_csv('../KineticsofLeakageOmpF/collatedIabs20210427.csv')
dfA = pd.read_csv('../KineticsofLeakageOmpF/collatedAabs20210427.csv')

In [8]:
dfI = dfI.iloc[:,1:]
dfA = dfA.iloc[:,1:]

In [9]:
dfA = dfA.fillna(0)
dfI = dfI.fillna(0)

# Remove small detected objects, most likely not GUVs

**check that the size is appropriate for the experiment**

In [10]:
# remove traces for GUVs below a certain area

App = 3.16 #um^2
Amin= 75*App*1e-12 #m^2

for col in dfI.columns:
    
    if col in dfA.columns:
        area = 1e-12*App*np.mean(dfA[col].iloc[:30])
        
        if area < Amin:
            dfI.pop(col)
            dfA.pop(col)

# Remove intensity curves with sudden drops
 These are likely from GUVs suddenly escaping the trap

In [11]:

def remove_burst_events(data,threshold = 0.3):
    
    for column in data.columns:
        
        if len(data[column][data[column] < threshold]) >=1:
            data.pop(column)
            

In [14]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(dfI[col].iloc[t0:])

In [13]:
# normalise intensities

for col in dfI.columns:
    dfI[col] = dfI[col]/np.mean(dfI[col][:10])
remove_burst_events(dfI)

# Smooth data

## Smoothing strategy

1. In experiments where there was relatively fast decay in intensity initially, but slows down (as exponential decays do..), yet there is still a lot of noise, it may be necessary to smooth different regions with different 'cutoff's.
When the intensity has dropped, the signal to noise drops. What's more during the slower period of intensity decay, there is increasing likelihood that some long timescale event will have happened to distort the data. This long timescale noise may be of a similar timescale to the 'signal' in the early period of rapid decay. 

2. The **solution** chosen here, is to have the user select a timepoint to bisect the time traces. The early period up until that timepoint is smoothed with the 1st user input smoothing parameter (usually between 11-21). Then the later period of slow decay is smoother with much larger smoothing settings (50-80). In order to ensure a smooth 'stitching' of these differently smoothed traces, we concatenate and repeat the smoothing with the smallest of the two specified smoothing parameters.

In [40]:
def smooth_decay(I,t0,t1=-1,smoothing_0 = 17,smoothing_1 = 81):
    
    
    if smoothing_0 > t1-t0 and t1 != -1:
        raise ValueError('Smoothing parameter cannot be greater than input data')
    
    smoothed_decay_0 = I.iloc[t0:t1]
    
    smoothed_decay_0 = savgol_filter(smoothed_decay_0,smoothing_0,1)
    
    if t1 != -1:
        
        smoothed_decay_1 = I.iloc[t1:]
        smoothed_decay_1  = savgol_filter(smoothed_decay_1,smoothing_1,1)
        
        smoothed_decay = np.concatenate((smoothed_decay_0,smoothed_decay_1))
    else:
        smoothed_decay = smoothed_decay_0
                               
    final_smoothed_decay = savgol_filter(smoothed_decay,smoothing_0,1)
    return final_smoothed_decay
                                  
                                    

In [41]:
#Check smoothing still fits data
#
num = 4
fig,axs = plt.subplots(1,num)

for i,col in enumerate(dfI.columns[np.random.randint(0,len(dfI.columns),num)]):
    
    axs[i].plot(dfI[col].iloc[t0:])
    axs[i].plot(smooth_decay(dfI[col],t0,t1))
    

# Use calibration inverse to convert I to [K]

In [42]:
dfC_dict = {}

# can't exploit numpy broadcasting

for col in dfI.columns:
    
    smoothed_decay = smooth_decay(dfI[col],t0,t1)
#     print(smoothed_decay)
#     normalised_sd = smoothed_decay/np.mean(smoothed_decay[:30])
    
    cs = []
    for I in smoothed_decay:
#         print(I)
        cs.append(fit_func(I,fit))
    
    
    dfC_dict[col] = cs
    
dfC = pd.DataFrame(dfC_dict)

3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3


0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1


1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


In [18]:
# filter concentration data by gradient
# if 

fig, axs = plt.subplots(1,2)

for col in dfC.columns:
    
    grad = np.gradient(dfC[col])
    
#     if len(grad[grad <= -0.0009])/len(grad) > 0.05:
#         dfC.pop(col)
#         continue
    axs[0].hist(grad,15)
    axs[1].plot(dfC[col])


In [13]:
# find representative background concentration variation

# tifs = glob('/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/260pM/*.tif')

# bgs = []

# fig, ax = plt.subplots(1,1)

# for tif in tifs:
    
#     vid = tf.TiffFile(tif)
    
#     frames = vid.asarray()
    
#     bg = np.median(frames,axis  =(1,2))
#     bg = np.array(bg)/np.max(bg)
#     bgs.append(bg)
    
    
#     ax.plot(bg)

# This confirms that the normalised background is the same for each chamber. Suggesting potassium concs are identical between chambers



In [131]:
# c_bg = []
# for x in bg[360+t0:]:
#     c_bg.append(inv(x))

In [132]:
# fig, axs = plt.subplots(1,2)


# smoothed_background = gaussian(c_bg,sigma)
# axs[0].plot(smoothed_background)
# axs[1].plot(bg[360+t0:])

# Find concentration difference

In [43]:
# calculating c_ext - c_i

dfdeltaC_dict = {}

for col in dfC:
    
    dfdeltaC_dict[col] = 1-dfC[col]
    
dfdeltaC = pd.DataFrame(dfdeltaC_dict)# drift in the background intensity over time, once the potassium has arrived, is causing a drifting predicted potassium concentration
# this is unlikely to be the case in reality.
# Therefore the best thing may be to just assume that after an initial 80 frame boundary region (20 mins) for this exp
# the potassium concentration externally reaches 1mM.


In [44]:
fig, axC = plt.subplots(1,1)

delete_cols = []

for col in dfdeltaC.columns:
    graddeltaC = np.gradient(dfdeltaC[col])
    
    
#     if len(graddeltaC[graddeltaC > 0]) > 10:
#         delete_cols.append(col)
#         continue
        
    axC.plot(dfdeltaC[col])

In [45]:
fig, axC = plt.subplots(1,1)

delete_cols = []

for col in dfdeltaC.columns:
    graddeltaC = np.gradient(dfdeltaC[col])
    
    
#     if len(graddeltaC[graddeltaC > 0]) > 1:
#         delete_cols.append(col)
#         continue
        
    axC.plot(dfdeltaC[col])

In [46]:
for col in delete_cols:
    dfdeltaC.pop(col)

# Determine the flux

In [47]:
# now calculate J

# J = (1/A)*dN/dt = R/3* dc_i/dt

dfJ_dict = {}


for col in dfdeltaC:
    
    if col in dfA:
        
        delta_c = np.gradient(dfC[col],dt)/60 #molm^-3s^-1
        
        A = np.mean(dfA[col].iloc[:30])*App*1e-12 #m^2
        
        R = np.sqrt(A/np.pi) #m
        
        dfJ_dict[col] = R/3*delta_c #molm^-2s^-1
        

dfJ = pd.DataFrame(dfJ_dict)



In [32]:
fig, axs = plt.subplots(1,3)
legend = []

for col in dfJ.columns:

    legend.append(col)
#     print(dfdeltaC)

    axs[0].scatter(dfdeltaC[col],dfJ[col]*1e10, marker = '+', s = 10)
    print(dfJ[col])

    axs[1].plot(dt*np.arange(0,len(dfC[col])),dfC[col])

    
#     gradients = []
#     win = 10
#     for i in range(dfJ[col].shape[0]-(win-1)):
        
#         gradient = np.mean(np.gradient(dfJ[col].iloc[i:i+win]))
#         gradients.append(gradient)
        
    
    axs[2].plot(np.gradient(dfJ[col],dfdeltaC[col]))
axs[0].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[1].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[0].set_xlabel('[K_ext]-[K_int] [mM]',fontsize = 18)

axs[0].set_ylabel('J [molm^-2s^-1]',fontsize = 18)

axs[1].set_ylabel('[K_int] [mM]',fontsize = 18)


axs[1].set_xlabel(' T [mins]',fontsize = 18)



#xticks

axs[0].set_xticks([0.5,0.75,1])
axs[0].set_xticklabels([0.5,0.75,1],fontsize = 15)

axs[1].set_xticks([0,20,40])
axs[1].set_xticklabels([0,20,40],fontsize = 15)

#yticks

axs[0].set_yticks([0,0.005,0.01,0.015])
axs[0].set_yticklabels([0,0.005,0.01,0.015],fontsize = 15)

axs[0].set_ylim([-0.0005,0.0155])
axs[1].set_yticks([0, 0.25,0.5])
axs[1].set_yticklabels([0,0.25,0.5],fontsize = 15)


# plt.rcParams['text.usetex'] = True

# def myticks(x,pos):

#     if x == 0: return "$0$"

#     exponent = int(np.log10(x))
#     coeff = x/10**exponent

#     return r"${:2.0f} \times 10^{{ {:2d} }}$".format(coeff,exponent)

# axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(myticks))

0      8.317596e-06
1      8.420413e-06
2      8.637645e-06
3      8.878939e-06
4      9.146102e-06
5      9.441085e-06
6      9.766001e-06
7      1.012314e-05
8      1.051498e-05
9      1.069851e-05
10     1.079304e-05
11     1.097868e-05
12     1.105023e-05
13     1.107871e-05
14     1.108236e-05
15     1.098628e-05
16     1.088626e-05
17     1.076522e-05
18     1.055438e-05
19     1.034039e-05
20     1.001419e-05
21     9.745836e-06
22     9.601824e-06
23     9.411570e-06
24     9.244518e-06
25     9.086152e-06
26     9.831855e-06
27     1.048528e-05
28     1.013521e-05
29     9.686426e-06
           ...     
251    1.229214e-07
252    1.229453e-07
253    1.229693e-07
254    1.229932e-07
255    1.230172e-07
256    1.230412e-07
257    1.230651e-07
258    1.230891e-07
259    1.231131e-07
260    1.231371e-07
261    1.231611e-07
262    1.231851e-07
263    1.232091e-07
264    1.232331e-07
265    1.232571e-07
266    1.232812e-07
267    1.233052e-07
268    1.233292e-07
269    1.233533e-07


0      1.709501e-06
1      1.712739e-06
2      1.719275e-06
3      1.725925e-06
4      1.732693e-06
5      1.739579e-06
6      1.746584e-06
7      1.753711e-06
8      1.760960e-06
9      1.646148e-06
10     1.453065e-06
11     1.338561e-06
12     1.239372e-06
13     1.115672e-06
14     1.019707e-06
15     9.539789e-07
16     8.695336e-07
17     7.663681e-07
18     6.603226e-07
19     5.089168e-07
20     3.277849e-07
21     2.024617e-07
22     1.307063e-07
23     9.471197e-08
24     1.034157e-07
25     1.277996e-07
26     1.027337e-07
27     4.940582e-08
28     3.770833e-08
29     6.948908e-08
           ...     
251    1.295865e-07
252    1.296308e-07
253    1.296751e-07
254    1.297194e-07
255    1.297638e-07
256    1.298083e-07
257    1.298528e-07
258    1.298974e-07
259    1.299420e-07
260    1.299866e-07
261    1.300313e-07
262    1.300760e-07
263    1.301208e-07
264    1.301657e-07
265    1.302106e-07
266    1.302555e-07
267    1.303005e-07
268    1.303455e-07
269    1.303906e-07


[Text(0, 0, '0'), Text(0, 0, '0.25'), Text(0, 0, '0.5')]

# Plot the Flux against concentration gradient



In [48]:
%matplotlib qt
figJ, axJ = plt.subplots(1,1)
figC, axC = plt.subplots(1,1)



def Jfit(deltac,k,c):
    return k*deltac + c

p0 = [1e-14,0]
Pparams = {}
cols = []
for i,col in enumerate(dfJ.columns):
    
    
    t = dt*np.arange(0,len(dfC[col]))
    
    J = dfJ[col]
    deltaC = dfdeltaC[col]
    
#     deltaC_bar = np.linspace(dfC[col].min(),1)
    
    params = curve_fit(Jfit,deltaC.iloc[deltaC.to_numpy().astype(float)>0.9],J.iloc[deltaC.to_numpy().astype(float)>0.9],p0)
    
    J_bar = Jfit(deltaC.iloc[deltaC.to_numpy().astype(float)>0.85],*params[0])
    P = params[0][0]
    dP = np.sqrt(params[1][0][0])
    print(dP/P, P)
    
    Pparams[col] = params
    
    if P < 0:
        
        continue
        
#     if len(J.iloc[(deltaC < 0.7).to_numpy()]) < 1:
#         continue
    
    cols.append(col)
    
    print(col)
    # plot fluxes
    axJ.scatter(deltaC,J,alpha = 0.25)
    axJ.plot(deltaC.iloc[deltaC.to_numpy().astype(float)>0.85],J_bar)
    
    #plot the Intensity vs time
    
    axC.plot(t,dfC[col])
 
#Flux plot params


axJ.set_xticks([0,0.5,1])
axJ.set_xlim([-0.05,1.05])
axJ.set_xticklabels([0,0.5,1],fontsize = 15)

axJ.set_xlabel('$[K]_{ext} - [K]_{in}$ [mM]',fontsize = 18)
axJ.set_yticks([0,2.5e-9,5e-9,7.5e-12,1e-8])
axJ.set_yticklabels([0,2.5,5,7.5,10],fontsize = 15)

axJ.set_ylim([-1e-9,11e-9])
#  
axJ.set_ylabel('J  [$10^{-12}$ mol$m^{-2}s^{-1}$]',fontsize = 18)
axJ.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)

axC.set_xticks([0,20,40])
axC.set_xticklabels([0,20,40],fontsize = 15)
axC.set_xlabel('Time [mins]',fontsize = 18)

axC.set_yticks([0,0.5,1])
axC.set_yticklabels([0,0.5,1],fontsize = 15)
axC.set_ylabel('Rel I',fontsize = 18 )
axC.set_ylim([-0.05,1.05])
axC.set_xlim([0,45])

# global params

axC.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)


0.07110591990909007 1.3508084682686827e-08
04001
0.04776329863283069 9.869691863959944e-09
04002
-0.05038476696527704 -2.2206495120751508e-08
0.175472379488015 7.582918727418552e-09
04006
0.02820908551670456 9.295700502691632e-09
04007
0.14500988539959617 7.427063209460463e-09
04008
0.07034241442086879 1.2986573568501118e-08
04009
0.06933656867101148 1.1365436748255248e-08
00002
0.06693855431492136 6.166686285026245e-09
00004
0.037294612816429165 4.1949329596237215e-08
00012
0.050931651079220444 3.2978479664282224e-08
00013
0.05956123800596582 3.977461179933839e-08
00015
0.06987219809848744 1.0395667093000026e-08
00016
0.2002498141969001 1.1243964265872291e-08
00019
0.06951337878710674 1.1671086401831366e-08
13001
0.028163663655305277 7.218973829250852e-09
13003
0.2586409837517168 1.3651068185858581e-08
13013
0.06852377234348489 8.644271450435199e-09
13014
0.2000093873875467 1.080642026888566e-08
13017
0.035738246763739656 9.47735872634132e-09
14002
0.0715517645548339 1.728059483696041

# Save permeability values

In [49]:
P_df = {'P ms^-1': [Pparams[key][0][0] for key in Pparams.keys()],
       'dP/P': [np.sqrt(Pparams[key][1][0][0])/Pparams[key][0][0] for key in Pparams.keys()],
       'A':[np.mean(dfA[key].iloc[:30]) for key in Pparams.keys()]}
        
P_identified = {key:[Pparams[key][0][0]] for key in Pparams.keys()}
P_identified = pd.DataFrame(P_identified)
P_df = pd.DataFrame(P_df)

In [50]:
P_df['P cms^-1'] = P_df['P ms^-1']*100

In [51]:
P_df.to_csv('Permeabilities_OmpF_20210427.csv')

# Save Flux vs deltaC data

In [28]:
dfJ.to_csv('Flux_OmpF_20210427.csv')
dfdeltaC.to_csv('deltaC_OmpF_20210427.csv')
P_identified.to_csv('PbyGUVId_OmpF_20210427.csv')